# Basis and reconstruction of 64 pixel image.
Joshua Stough
DIP

Decompose and partially reconstruct according to the Haar
basis.



In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
from waveletUtil import *
from ipywidgets import VBox, FloatSlider

In [21]:
print('haarReconstructionDemo: Works only on 64 pixel images...')

# I = plt.imread('surprise.png')
I = plt.imread('hello.png')
I = I[...,:3].copy()
I

haarReconstructionDemo: Works only on 64 pixel images...


array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       ...,

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)

In [3]:
from skimage.transform import rescale
plt.figure()
sI = rescale(I, 1/16, order = 1, anti_aliasing=False, multichannel=True)
plt.imshow(sI)
sI.shape

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(32, 32, 3)

In [4]:
s = 32

In [5]:
# H is the 8x8 Haar matrix
# H = np.eye(8) # For fun comparison.
# H = makeRandomBasis(8)
# H = makeKLTBasis(I, 8)
# H = makeDCTMatrix(8)
H = makeHaarMatrix(s)

In [12]:
#The transform image, an image of coefficients wrt the Haar basis.
CT = np.zeros(sI.shape)
for chan in range(3):
    CT[...,chan] = np.matmul(H, np.matmul(sI[...,chan], H.transpose()))


#RI will represent the reconstructed image as we add back more
#Haar patterns
RI = np.zeros(sI.shape) #Still should be 8x8
sI.shape

(32, 32, 3)

In [13]:
# We're going to reconstruct according to distance from the
# 0,0 (the first Haar basis, average calculator). Notice,
# this order is independent of the actual image data.
xs = np.meshgrid(np.arange(s), np.arange(s), indexing='ij')
coords = np.concatenate([np.expand_dims(c, axis=1) for c in
                         [x.ravel() for x in xs]], axis=1)
dists = np.sum(coords*coords, axis=1)
darg = np.argsort(dists) #sorts in increasing order
darg.min()

0

In [14]:
from ipywidgets import VBox, IntSlider

In [15]:
def make_image(change):
    global coords, H, CT
    ReI = np.zeros(sI.shape)
    Bij = np.outer(H[0, :], H[0, :])
    for ind in range(change):
        i,j = coords[ind] #coords[darg[x]] if x in range(len(darg))

    #Construct that Haar basis and display it
        Bij = np.outer(H[i, :], H[j, :])
    
    # Add the amount of that basis that was in the original image to
    # the running total, or reconstruction.
    # RI = RI + T[i, j] * Bij #2D
        for chan in range(3):
            ReI[...,chan] += CT[i,j,chan]*Bij
    return Bij, ReI

In [20]:
base, i = make_image(3)
i

array([[[0.89859069, 0.95677084, 0.93342525],
        [0.89859069, 0.95677084, 0.93342525],
        [0.89859069, 0.95677084, 0.93342525],
        ...,
        [0.93012408, 0.88063726, 0.84747243],
        [0.93012408, 0.88063726, 0.84747243],
        [0.93012408, 0.88063726, 0.84747243]],

       [[0.89859069, 0.95677084, 0.93342525],
        [0.89859069, 0.95677084, 0.93342525],
        [0.89859069, 0.95677084, 0.93342525],
        ...,
        [0.93012408, 0.88063726, 0.84747243],
        [0.93012408, 0.88063726, 0.84747243],
        [0.93012408, 0.88063726, 0.84747243]],

       [[0.89859069, 0.95677084, 0.93342525],
        [0.89859069, 0.95677084, 0.93342525],
        [0.89859069, 0.95677084, 0.93342525],
        ...,
        [0.93012408, 0.88063726, 0.84747243],
        [0.93012408, 0.88063726, 0.84747243],
        [0.93012408, 0.88063726, 0.84747243]],

       ...,

       [[0.89859069, 0.95677084, 0.93342525],
        [0.89859069, 0.95677084, 0.93342525],
        [0.89859069, 0

In [19]:
basis, init = make_image(1)

plt.ioff()
plt.clf()

# https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#FloatLogSlider
slider = IntSlider(
    orientation='horizontal',
    value=1,
    min=1,
    max=s**2,
    step=1,
    description='Base element'
)

fig_args = {'num':' ', 'frameon':True}
fig, ax = plt.subplots(1,2, figsize=(10,3), **fig_args) 

# display artists I'll update
adisp = ax[0].imshow(basis, cmap='gray', vmin=-0.5, vmax=0.5)
hdisp = ax[1].imshow(init, cmap='magma', vmin=0, vmax=1)

def update_image(change):    
    print(change)
    new_base, new_re = make_image(change.new)
    hdisp.set_array(new_re)
    # need to reset the color limits each time since gaus_h is changing a lot. 
    hdisp.set_clim(0,1) 
    
    adisp.set_array(new_base)
    
    fig.canvas.draw()
    fig.canvas.flush_events()

slider.observe(update_image, names='value')

VBox([slider, fig.canvas])